In [ ]:
from IPython.display import HTML
import csv
import requests

import optuna

import pandas as pd
import numpy as np
import pickle
import weakref

import os
import sys

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


from keras.layers import LeakyReLU
from keras.models import Model
from keras.layers import *

from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt

# Plot the generator model architecture
from tensorflow.keras.utils import plot_model



import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
Features = pd.read_csv("filtered_data_After_blanced.csv")
Features.head()

In [ ]:
Features.tail()

In [ ]:
X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

In [ ]:
# As this is a multiclass classification problem onehotencoding our Y.
encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

In [ ]:
# splitting data
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)

In [ ]:
# making our data compatible to model.
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape


In [ ]:
# Input shape for 1D convolutional model
input_shape = x_train.shape[1], 1

# Build the generator model
generator = Sequential()
generator.add(Dense(256, input_shape=input_shape))
generator.add(LeakyReLU(alpha=0.01))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.01))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.01))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(2048))
generator.add(LeakyReLU(alpha=0.01))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(4096))
generator.add(LeakyReLU(alpha=0.01))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1024))
generator.add(LeakyReLU(alpha=0.01))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(512))
generator.add(LeakyReLU(alpha=0.01))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(256))
generator.add(LeakyReLU(alpha=0.01))
generator.add(BatchNormalization(momentum=0.8))
generator.add(Dense(1, activation='tanh'))

# Plot the generator model architecture
plot_model(generator, to_file='generator.png', show_shapes=True, show_layer_names=True)

# Build the discriminator model
discriminator = Sequential()
discriminator.add(Conv1D(256, kernel_size=5, strides=2, input_shape=input_shape, padding="same"))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(Dropout(0.25))
discriminator.add(Conv1D(128, kernel_size=5, strides=2, padding="same"))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(BatchNormalization(momentum=0.8))
discriminator.add(Dropout(0.25))
discriminator.add(Flatten())
discriminator.add(Dense(7, activation='softmax'))

# Plot the discriminator model architecture
plot_model(discriminator, to_file='discriminator.png', show_shapes=True, show_layer_names=True)

# Combine the generator and discriminator to create the GAN
gan_input = Input(shape=input_shape)
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(inputs=gan_input, outputs=gan_output)

# Plot the GAN model architecture
plot_model(gan, to_file='gan.png', show_shapes=True, show_layer_names=True)

# Compile the GAN
gan.compile(loss='categorical_crossentropy',metrics = ["accuracy"], optimizer=Adam(lr=0.0002, beta_1=0.5))

generator.summary()
discriminator.summary()
gan.summary()



In [ ]:
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.8, patience=5, min_lr=0.00001)

history=gan.fit(x_train, y_train, batch_size=64, epochs=250 ,validation_data=(x_test, y_test), callbacks=[rlrp])

In [ ]:
print("Accuracy of our model on test data : " , gan.evaluate(x_test,y_test)[1]*100 , "%")

epochs = [i for i in range(250)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
test_acc = history.history['val_accuracy']
test_loss = history.history['val_loss']

fig.set_size_inches(40,20)
ax[0].plot(epochs , train_loss , label = 'Training Loss')
ax[0].plot(epochs , test_loss , label = 'Testing Loss')
ax[0].set_title('Training & Testing Loss')
ax[0].legend()
ax[0].set_xlabel("Epochs")

ax[1].plot(epochs , train_acc , label = 'Training Accuracy')
ax[1].plot(epochs , test_acc , label = 'Testing Accuracy')
ax[1].set_title('Training & Testing Accuracy')
ax[1].legend()
ax[1].set_xlabel("Epochs")
plt.show()
gan.save("GanModelFileH5.h5")
print("Model saved to disk")


In [ ]:
# predicting on test data.
pred_test = gan.predict(x_test)
y_pred = encoder.inverse_transform(pred_test)

y_test = encoder.inverse_transform(y_test)

In [ ]:
df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred.flatten()
df['Actual Labels'] = y_test.flatten()

df.head(30)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize = (12, 10))
cm = pd.DataFrame(cm , index = [i for i in encoder.categories_] , columns = [i for i in encoder.categories_])
sns.heatmap(cm, linecolor='white', cmap='Blues', linewidth=1, annot=True, fmt='')
plt.title('Confusion Matrix', size=20)
plt.xlabel('Predicted Labels', size=14)
plt.ylabel('Actual Labels', size=14)
plt.show()

In [ ]:
print(classification_report(y_test, y_pred))